### Merge Upstream Downstream with FAO names 

* Purpose of script: Create a shapefile and csv file with both the upstream / downstream relation and the FAO basin names
* Author: Rutger Hofste
* Kernel used: python35
* Date created: 20170829

In [1]:
S3_INPUT_PATH_FAO ="s3://wri-projects/Aqueduct30/processData/Y2017M08D25_RH_spatial_join_FAONames_V01/output/"
S3_INPUT_PATH_DOWNSTREAM = "s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output/"
S3_INPUT_PATH_HYBAS = "s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/"

INPUT_FILE_NAME_FAO = "hybas_lev06_v1c_merged_fiona_withFAO_V01.csv"
INPUT_FILE_NAME_DOWNSTREAM = "hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv"
INPUT_FILE_NAME_HYBAS = "hybas_lev06_v1c_merged_fiona_V01.shp"

EC2_INPUT_PATH = "Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/"
EC2_OUTPUT_PATH = "Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

OUTPUT_FILE_NAME = "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01"

S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

In [2]:
!rm -r {EC2_INPUT_PATH}
!rm -r {EC2_OUTPUT_PATH}

!mkdir -p {EC2_INPUT_PATH}
!mkdir -p {EC2_OUTPUT_PATH}

In [3]:
!aws s3 cp {S3_INPUT_PATH_FAO} {EC2_INPUT_PATH} --recursive 

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D25_RH_spatial_join_FAONames_V01/output/hybas_lev06_v1c_merged_fiona_withFAO_V01.csv to Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev06_v1c_merged_fiona_withFAO_V01.csv


In [4]:
!aws s3 cp {S3_INPUT_PATH_DOWNSTREAM} {EC2_INPUT_PATH} --recursive 

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D23_RH_Downstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv to Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev06_v1c_merged_fiona_upstream_downstream_V01.csv


In [5]:
!aws s3 cp {S3_INPUT_PATH_HYBAS} {EC2_INPUT_PATH} --recursive --exclude *.tif

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev00_v1c_merged_fiona_V01.cpg to Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev00_v1c_merged_fiona_V01.cpg
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev00_v1c_merged_fiona_V01.prj to Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev00_v1c_merged_fiona_V01.prj
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.prj to Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev06_v1c_merged_fiona_V01.prj
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merged_fiona_V01.dbf to Y2017M08D29_RH_Merge_FAONames_Upstream_V01/input/hybas_lev06_v1c_merged_fiona_V01.dbf
download: s3://wri-projects/Aqueduct30/processData/Y2017M08D02_RH_Merge_HydroBasins_V01/output/hybas_lev06_v1c_merge

In [6]:
import os
if 'GDAL_DATA' not in os.environ:
    os.environ['GDAL_DATA'] = r'/usr/share/gdal/2.1'
from osgeo import gdal,ogr,osr
'GDAL_DATA' in os.environ
# If false, the GDAL_DATA variable is set incorrectly. You need this variable to obtain the spatial reference
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import time
%matplotlib notebook

In [7]:
dfFAO = pd.read_csv(os.path.join(EC2_INPUT_PATH,INPUT_FILE_NAME_FAO))

In [8]:
dfFAO.head()

,PFAF_ID,SUB_NAME,MAJ_NAME,FAOid_copy
0,111011,['Wadi El Naqa'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190312']
1,111012,['Egyptian east coast'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190313']
2,111013,['Egyptian east coast'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190313']
3,111014,['Egyptian east coast'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190313']
4,111015,['Egyptian east coast'],"['Africa, Red Sea - Gulf of Aden Coast']",['MAJ_BAS_7019_SUB_BASE_0190313']


In [9]:
dfFAO.shape

(16396, 4)

In [10]:
dfDownstream = pd.read_csv(os.path.join(EC2_INPUT_PATH,INPUT_FILE_NAME_DOWNSTREAM))

In [11]:
dfDownstream.head()

,HYBAS_ID2,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,...,COAST,ORDER,SORT,Upstream_HYBAS_IDs,Upstream_PFAF_IDs,Downstream_HYBAS_IDs,Downstream_PFAF_IDs,Basin_HYBAS_IDs,Basin_PFAF_IDs,NEXT_SINK_PFAF
0,6060000010,0,6060000010,0,6060000010,6060000010,0.0,0.0,4317.4,4317.4,...,1,0,1,[],[],[],[],[6060000010],[611001],611001.0
1,6060000200,1,6060000200,0,6060000200,6060000200,0.0,0.0,35995.5,35996.7,...,0,1,2,[],[],[],[],[6060000200],[611002],611002.0
2,6060000210,2,6060000210,0,6060000210,6060000210,0.0,0.0,443.9,443.9,...,1,0,3,[],[],[],[],[6060000210],[611003],611003.0
3,6060000240,3,6060000240,0,6060000240,6060000240,0.0,0.0,2186.3,2186.3,...,0,1,4,[],[],[],[],[6060000240],[611004],611004.0
4,6060000250,4,6060000250,0,6060000250,6060000250,0.0,0.0,6533.8,6533.8,...,1,0,5,[],[],[],[],[6060000250],[611005],611005.0


In [12]:
list(dfDownstream)

['HYBAS_ID2',
 'Unnamed: 0',
 'HYBAS_ID',
 'NEXT_DOWN',
 'NEXT_SINK',
 'MAIN_BAS',
 'DIST_SINK',
 'DIST_MAIN',
 'SUB_AREA',
 'UP_AREA',
 'PFAF_ID',
 'ENDO',
 'COAST',
 'ORDER',
 'SORT',
 'Upstream_HYBAS_IDs',
 'Upstream_PFAF_IDs',
 'Downstream_HYBAS_IDs',
 'Downstream_PFAF_IDs',
 'Basin_HYBAS_IDs',
 'Basin_PFAF_IDs',
 'NEXT_SINK_PFAF']

In [13]:
dfDownstream.shape

(16397, 22)

In [14]:
gdfHybas = gpd.read_file(os.path.join(EC2_INPUT_PATH,INPUT_FILE_NAME_HYBAS))

In [15]:
gdfOut = gdfHybas.merge(dfDownstream, on='PFAF_ID',how="inner")

In [16]:
gdfOut = gdfOut.merge(dfFAO,on='PFAF_ID',how="inner")

In [17]:
dfOut = gdfOut.drop('geometry',1)

In [18]:
dfOut.head()

,COAST_x,DIST_MAIN_x,DIST_SINK_x,ENDO_x,HYBAS_ID_x,MAIN_BAS_x,NEXT_DOWN_x,NEXT_SINK_x,ORDER_x,PFAF_ID,...,Upstream_HYBAS_IDs,Upstream_PFAF_IDs,Downstream_HYBAS_IDs,Downstream_PFAF_IDs,Basin_HYBAS_IDs,Basin_PFAF_IDs,NEXT_SINK_PFAF,SUB_NAME,MAJ_NAME,FAOid_copy
0,1,0.0,0.0,0,6060000010,6060000010,0,6060000010,0,611001,...,[],[],[],[],[6060000010],[611001],611001.0,"['Archipielago de San Blas Coast', 'Altrato 1']","['Caribbean Coast', 'Caribbean Coast']","['MAJ_BAS_3001_SUB_BASE_0001002', 'MAJ_BAS_300..."
1,0,0.0,0.0,0,6060000200,6060000200,0,6060000200,1,611002,...,[],[],[],[],[6060000200],[611002],611002.0,"['Altrato 1', 'Sucio', 'Altrato 2']","['Caribbean Coast', 'Caribbean Coast', 'Caribb...","['MAJ_BAS_3001_SUB_BASE_0001003', 'MAJ_BAS_300..."
2,1,0.0,0.0,0,6060000210,6060000210,0,6060000210,0,611003,...,[],[],[],[],[6060000210],[611003],611003.0,"['Altrato 1', 'Golfo del Darien Coast']","['Caribbean Coast', 'Caribbean Coast']","['MAJ_BAS_3001_SUB_BASE_0001003', 'MAJ_BAS_300..."
3,0,0.0,0.0,0,6060000240,6060000240,0,6060000240,1,611004,...,[],[],[],[],[6060000240],[611004],611004.0,['Golfo del Darien Coast'],['Caribbean Coast'],['MAJ_BAS_3001_SUB_BASE_0001006']
4,1,0.0,0.0,0,6060000250,6060000250,0,6060000250,0,611005,...,[],[],[],[],[6060000250],[611005],611005.0,"['Golfo del Darien Coast', 'Sinu']","['Caribbean Coast', 'Caribbean Coast']","['MAJ_BAS_3001_SUB_BASE_0001006', 'MAJ_BAS_300..."


In [19]:
dfOutSimple = dfOut["PFAF_ID"]

In [20]:
gdfOutSimple = gpd.GeoDataFrame(dfOutSimple, geometry=gdfOut.geometry)

In [21]:
gdfOutSimple.to_file(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILE_NAME+".shp"))

In [22]:
dfOut.to_csv(os.path.join(EC2_OUTPUT_PATH,OUTPUT_FILE_NAME+".csv"))

In [23]:
!aws s3 cp {EC2_OUTPUT_PATH} {S3_OUTPUT_PATH} --recursive

upload: Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.prj to s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.prj
upload: Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.cpg to s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.cpg
upload: Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.dbf to s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.dbf
upload: Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.csv to s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_